# Programming Assignment No. 5: Lasso Algorithm Implementation
#                                          (Part II: Coordinate Descent)

In this notebook, you will implement your very own **lasso** solver via coordinate descent. You will:
* Write a function to normalize features
* Implement coordinate descent for the **lasso** algorithm
* Explore effects of $ L_{1} $ penalty

# Fire up graphlab create

Make sure you have the latest version of graphlab (>= 1.7)

In [1]:
import graphlab

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
sales = graphlab.SFrame('kc_house_data.gl/')
# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to int, before using it below
sales['floors'] = sales['floors'].astype(int) 

This non-commercial license of GraphLab Create for academic use is assigned to s201369650@kfupm.edu.sa and will expire on October 03, 2018.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: C:\Users\omarw\AppData\Local\Temp\graphlab_server_1510422753.log.0


If we want to do any "feature engineering" like creating new features or adjusting existing ones we should do this directly using the SFrames as seen in the first notebook of the programming assignment on ** Poynomial Regression **. For this notebook, however, we will work with the existing features.

# Import useful functions from previous notebook

As in the programming assignment on ** Poynomial Regression **, we convert the SFrame into a 2D Numpy array. Copy and paste `get_num_data()` from the notebook in the programming assignment on ** Poynomial Regression **.

In [3]:
import numpy as np # note this allows us to refer to numpy as np instead 

In [4]:
def get_numpy_data(data_sframe, features, output):
    data_sframe['constant'] = 1 # this is how you add a constant column to an SFrame
    # add the column 'constant' to the front of the features list so that we can extract it along with the others:
    features = ['constant'] + features # this is how you combine two lists
    # select the columns of data_SFrame given by the features list into the SFrame features_sframe (now including constant):
    features_sframe = data_sframe[features]
    
    # the following line will convert the features_SFrame into a numpy matrix:
    feature_matrix = features_sframe.to_numpy()
    # assign the column of data_sframe associated with the output to the SArray output_sarray
    output_sarray = data_sframe[output]

    # the following will convert the SArray into a numpy array by first converting it to a list
    output_array = output_sarray.to_numpy()
    return(feature_matrix, output_array)

Also, copy and paste the `predict_output()` function to compute the predictions for an entire matrix of features given the matrix and the weights:

In [16]:
def predict_output(feature_matrix, weights):
    # assume feature_matrix is a numpy matrix containing the features as columns and weights is a corresponding numpy array
    # create the predictions vector by using np.dot()
    predictions = np.dot(feature_matrix, weights)
    return(predictions)

# Normalize features
In the house dataset, features vary wildly in their relative magnitude: `sqft_living` is very large overall compared to `bedrooms`, for instance. As a result, weight for `sqft_living` would be much smaller than weight for `bedrooms`. This is problematic because "small" weights are dropped first as `l1_penalty` goes up. 

To give equal considerations for all features, we need to **normalize features** as discussed in the lectures: we divide each feature by its 2-norm so that the transformed feature has norm 1.

Let's see how we can do this normalization easily with Numpy: let us first consider a small matrix.

In [17]:
X = np.array([[3.,5.,8.],[4.,12.,15.]])
print X

[[  3.   5.   8.]
 [  4.  12.  15.]]


Numpy provides a shorthand for computing 2-norms of each column:

In [18]:
norms = np.linalg.norm(X, axis=0) # gives [norm(X[:,0]), norm(X[:,1]), norm(X[:,2])]
print norms

[  5.  13.  17.]


To normalize, apply element-wise division:

In [10]:
print X / norms # gives [X[:,0]/norm(X[:,0]), X[:,1]/norm(X[:,1]), X[:,2]/norm(X[:,2])]

[[ 0.6         0.38461538  0.47058824]
 [ 0.8         0.92307692  0.88235294]]


Using the shorthand we just covered, write a short function called `normalize_features(feature_matrix)`, which normalizes columns of a given feature matrix. The function should return a pair `(normalized_features, norms)`, where the second item contains the norms of original features. As discussed in the lectures, we will use these norms to normalize the test data in the same way as we normalized the training data. 

In [11]:
def normalize_features(feature_matrix):
    norms = np.linalg.norm(feature_matrix, axis=0)
    normalized_features = feature_matrix / norms
    return (normalized_features, norms)

To test the function, run the following:

In [12]:
features, norms = normalize_features(np.array([[3.,6.,9.],[4.,8.,12.]]))
print features
# should print
# [[ 0.6  0.6  0.6]
#  [ 0.8  0.8  0.8]]
print norms
# should print
# [5.  10.  15.]

[[ 0.6  0.6  0.6]
 [ 0.8  0.8  0.8]]
[  5.  10.  15.]


# Implementing Coordinate Descent with normalized features

We seek to obtain a sparse set of weights by minimizing the ** lasso ** cost function
$$
\sum_{i = 1}^{m} \left( y^{(i}) - \mathbf{w}^{T} x^{(i)} \right)^{2} + \lambda \left( \left| w_{1} \right| + \left| w_{2} \right| + \cdots + \left| w_{n} \right| \right)
$$

** Note: ** We do not include $ w_{0} $ in the $ L_{1} penalty term. We never want to push the intercept to zero!

The absolute value sign makes the cost function non-differentiable, so simple gradient descent is not viable (you would need to implement a method called subgradient descent). Instead, we will use **coordinate descent**: at each iteration, we will fix all weights but weight `i` and find the value of weight `i` that minimizes the objective. That is, we look 

$$
\argmin_{w_{j}} \left( \sum_{i = 1}^{m} \left( y^{(i}) - \mathbf{w}^{T} x^{(i)} \right)^{2} + \lambda \left( \left| w_{1} \right| + \left| w_{2} \right| + \cdots + \left| w_{n} \right| \right) \right)
$$

where all weights other than $ w_{j} $ are held to be constant. We will optimize one $ w_{j} $ at a time, circling through the weights multiple times.  
  1. Pick a coordinate $ j $
  2. Compute $ w_{j} $ that minimizes the cost function above
  3. Repeat Steps 1 and 2 for all coordinates, multiple times

For this notebook, we use **cyclical coordinate descent with normalized features**, where we cycle through coordinates 0 to (n-1) in order, and assume the features were normalized as discussed above. The formula for optimizing each coordinate is as follows:
```
       ┌ (ro[j] + lambda/2)     if ro[j] < -lambda/2
w[j] = ├ 0                      if -lambda/2 <= ro[j] <= lambda/2
       └ (ro[j] - lambda/2)     if ro[j] > lambda/2
```
where
```
ro[j] = SUM[ [feature_j]*(output - prediction + w[j]*[feature_j]) ].
```

Note that we do not regularize the weight of the constant feature (intercept) $ w_{0} $, so, for this weight, the update is simply: 
$$ 
w_{0} = \rho_{0} 
$$

## Effect of $L_{1} $ penalty

Let us consider a simple model with 2 features:

In [13]:
simple_features = ['sqft_living', 'bedrooms']
my_output = 'price'
(simple_feature_matrix, output) = get_numpy_data(sales, simple_features, my_output)

Do not forget to normalize features:

In [14]:
simple_feature_matrix, norms = normalize_features(simple_feature_matrix)

We assign some random set of initial weights and inspect the values of $ \rho{j} $:

In [15]:
weights = np.array([1., 4., 1.])

Use `predict_output()` to make predictions on this data.

In [19]:
prediction = predict_output(simple_feature_matrix, weights)

Compute the values of $ \rho{j} $ for each feature in this simple model, using the formula given above, using the formula:
```
ro[j] = SUM[ [feature_j]*(output - prediction + w[j]*[feature_j]) ]
```

*Hint: You can get a Numpy vector for feature_i using:*
```
simple_feature_matrix[:,j]
```

In [29]:
ro=[]
for j in range(0, len(weights)):
    roJ = np.dot(simple_feature_matrix[:,j],( output - prediction + weights[j]*simple_feature_matrix[:,j]))
    ro.append(roJ)
print ro[1] , ro[2]

87939470.773 80966698.676


**Analysis Question No. 1:** Recall that, whenever $ \rho_{j} $ falls between `-l1_penalty/2` and `l1_penalty/2`, the corresponding weight $ w_{j} $ is sent to zero. Now suppose we were to take one step of coordinate descent on either feature 1 or feature 2. What range of values of `l1_penalty` **would not** set $ w_{1} $ zero, but **would** set $ w_{2} $ to zero, if we were to take a step in that coordinate? 

In [25]:
print "%.f <= lambda < %.f" % (2*ro[2], 2*ro[1])

161933397 <= lambda < 175878942


**Analysis Question No. 2:** What range of values of `l1_penalty` would set **both** $ w_{1} $  and $ w_{2} $ to zero, if we were to take a step in that coordinate? 

In [28]:
print "-lambda <= %.f <= lambda" %(ro[1] + ro[2])

-lambda <= 168906169 <= lambda


So we can say that $ \rho_{j} $ quantifies the significance of the j-th feature: the larger $ \rho_{j} $ is, the more likely it is for the i-th feature to be retained.

## Single Coordinate Descent Step

Using the formula above, implement coordinate descent that minimizes the cost function over a single feature $ j $. Note that the intercept ($ w_{0} $) is not regularized. The function should accept feature matrix, output, current weights, $ L_{1} $ penalty, and index of feature to optimize over. The function should return new weight for feature $ j $.

In [32]:
def lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty):
    # compute prediction
    prediction = predict_output(feature_matrix, weights)
    # compute ro[i] = SUM[ [feature_i]*(output - prediction + weight[i]*[feature_i]) ]
    ro_i =  np.dot(feature_matrix[:,i],( output - prediction + weights[i]*feature_matrix[:,i]))

    if i == 0: # intercept -- do not regularize
        new_weight_i = ro_i 
    elif ro_i < -l1_penalty/2.:
        new_weight_i = ro_i + l1_penalty/2.
    elif ro_i > l1_penalty/2.:
        new_weight_i = ro_i - l1_penalty/2.
    else:
        new_weight_i = 0.
    
    return new_weight_i

To test the function, run the following cell:

In [33]:
# should print 0.425558846691
import math
print lasso_coordinate_descent_step(1, np.array([[3./math.sqrt(13),1./math.sqrt(10)],[2./math.sqrt(13),3./math.sqrt(10)]]), 
                                   np.array([1., 1.]), np.array([1., 4.]), 0.1)

0.425558846691


## Cyclical coordinate descent 

Now that we have a function that optimizes the cost function over a single coordinate, let us implement cyclical coordinate descent where we optimize coordinates 0, 1, ..., (n-1) in order and repeat.

** Question: ** When do we know to stop? 

** Answer: ** Each time we scan all the coordinates (features) once, we measure the change in weight for each coordinate. If no coordinate changes by more than a specified threshold, we stop.

For each iteration:
1. As you loop over features in order and perform coordinate descent, measure how much each coordinate changes.
2. After the loop, if the maximum change across all coordinates is falls below the tolerance, stop. Otherwise, go back to step 1.

Return weights

**Important Note: ** When computing a new weight for coordinate i, make sure to incorporate the new weights for coordinates 0, 1, ..., j-1. One good way is to update your weights variable in-place. See following pseudocode for illustration.**
```
for i in range(len(weights)):
    old_weights_i = weights[i] # remember old value of weight[i], as it will be overwritten
    # the following line uses new values for weight[0], weight[1], ..., weight[i-1]
    #     and old values for weight[i], ..., weight[d-1]
    weights[i] = lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty)
    
    # use old_weights_i to compute change in coordinate
    ...
```

In [44]:
def lasso_cyclical_coordinate_descent(feature_matrix, output, initial_weights, l1_penalty, tolerance):
    weights = initial_weights
    max_change = tolerance + 1
    while(max_change > tolerance):
        max_change = 0
        for i in range(len(weights)):
            old_weights_i = weights[i]
            weights[i] = lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty)
            change = abs(old_weights_i - weights[i])
            max_change = max(max_change,change)
    return weights

Using the following parameters, learn the weights on the sales dataset. 

In [45]:
simple_features = ['sqft_living', 'bedrooms']
my_output = 'price'
initial_weights = np.zeros(3)
l1_penalty = 1e7
tolerance = 1.0

First create a normalized version of the feature matrix, `normalized_simple_feature_matrix`.

In [46]:
(simple_feature_matrix, output) = get_numpy_data(sales, simple_features, my_output)
(normalized_simple_feature_matrix, simple_norms) = normalize_features(simple_feature_matrix) # normalize features

Then, run your implementation of the ** lasso ** coordinate descent:

In [53]:
weights = lasso_cyclical_coordinate_descent(normalized_simple_feature_matrix, output,
                                            initial_weights, l1_penalty, tolerance)

**Analysis Question No. 3:** What is the RSS of the learned model on the normalized dataset? 

**Hint:** Use the normalized feature matrix when you make predictions.

**Analysis Question No. 4:** Which features had weight zero at convergence?

In [48]:
def compute_rss(output, feature_matrix, weights):
    error = output - predict_output(feature_matrix, weights)
    rss = (error**2.).sum()
    return rss

In [52]:
print compute_rss(output, normalized_simple_feature_matrix, weights)
print weights

1.63049248148e+15
[ 21624998.36636292  63157246.78545423         0.        ]


# Evaluating the lasso fit with more features

Let us split the sales dataset into training and test sets.

In [54]:
train_data,test_data = sales.random_split(.8,seed=0)

Let us consider the following set of features.

In [55]:
all_features = ['bedrooms',
                'bathrooms',
                'sqft_living',
                'sqft_lot',
                'floors',
                'waterfront', 
                'view', 
                'condition', 
                'grade',
                'sqft_above',
                'sqft_basement',
                'yr_built', 
                'yr_renovated']

First, create a normalized feature matrix from the ** training ** data with these features. 

** Note: ** Make you store the norms for the normalization, since we will use them later.

In [60]:
(feature_matrix, output) = get_numpy_data(train_data, all_features, 'price')
(normalized_feature_matrix, norms) = normalize_features(feature_matrix)

First, learn the weights with `l1_penalty=1e7`, on the ** training ** data. Initialize weights to all zeros, and set the `tolerance=1`.  Call resulting weights `weights1e7`, you will need them later.

In [65]:
weights1e7 = lasso_cyclical_coordinate_descent(normalized_feature_matrix, output,
                                            initial_weights=np.zeros(len(all_features)+1), l1_penalty=1e7, tolerance=1)

**Analysis Question No. 5:** What features had non-zero weight in this case?

In [68]:
def print_weights(weights, features):
    names = list(features)
    names.insert(0, 'w0')
    for i in range(len(weights)):
        print "%13s %f" % (names[i], weights[i])
print_weights(weights1e7, all_features)

           w0 24429600.609333
     bedrooms 0.000000
    bathrooms 0.000000
  sqft_living 48389174.352280
     sqft_lot 0.000000
       floors 0.000000
   waterfront 3317511.162720
         view 7329961.984896
    condition 0.000000
        grade 0.000000
   sqft_above 0.000000
sqft_basement 0.000000
     yr_built 0.000000
 yr_renovated 0.000000


Next, learn the weights with `l1_penalty=1e8`, on the ** training data **. Initialize weights to all zeros, and set the `tolerance=1`.  Call resulting weights `weights1e8`, you will need them later.

In [67]:
weights1e8 = lasso_cyclical_coordinate_descent(normalized_feature_matrix, output,
                                            initial_weights=np.zeros(len(all_features)+1), l1_penalty=1e8, tolerance=1)

**Analysis Question No. 6:** What features had non-zero weight in this case?

In [69]:
print_weights(weights1e8, all_features)

           w0 71114625.752809
     bedrooms 0.000000
    bathrooms 0.000000
  sqft_living 0.000000
     sqft_lot 0.000000
       floors 0.000000
   waterfront 0.000000
         view 0.000000
    condition 0.000000
        grade 0.000000
   sqft_above 0.000000
sqft_basement 0.000000
     yr_built 0.000000
 yr_renovated 0.000000


Finally, learn the weights with `l1_penalty=1e4`, on the ** training ** data. Initialize weights to all zeros, and set the `tolerance=5e5`.  Call resulting weights `weights1e4`, you will need them later. 

** Note: ** This case will take quite a bit longer to converge than the others above!

In [72]:
weights1e4 = lasso_cyclical_coordinate_descent(normalized_feature_matrix, output,
                                            initial_weights=np.zeros(len(all_features)+1), l1_penalty=1e4, tolerance=5e5)

**Analysis Question No. 7:** What features had non-zero weight in this case?

In [73]:
print_weights(weights1e4, all_features)

           w0 77779073.912652
     bedrooms -22884012.250234
    bathrooms 15348487.080900
  sqft_living 92166869.698831
     sqft_lot -2139328.082428
       floors -8818455.544095
   waterfront 6494209.733107
         view 7065162.050532
    condition 4119079.210068
        grade 18436483.526188
   sqft_above -14566678.545143
sqft_basement -5528348.751794
     yr_built -83591746.207305
 yr_renovated 2784276.460129


## Rescaling learned weights

Recall that we normalized our feature matrix, before learning the weights.  To use these weights on a test set, we must normalize the test data in the same way.

Alternatively, we can rescale the learned weights to include the normalization, so we never have to worry about normalizing the ** test ** data: 

In this case, we must scale the resulting weights so that we can make predictions with *original* features:
 1. Store the norms of the original features to a vector called `norms`:
```
features, norms = normalize_features(features)
```
 2. Run Lasso on the normalized features and obtain a `weights` vector
 3. Compute the weights for the original features by performing element-wise division, i.e.
```
weights_normalized = weights / norms
```
Now, we can apply `weights_normalized` to the test data, without normalizing it!

Create a normalized version of each of the weights learned above. (`weights1e4`, `weights1e7`, `weights1e8`).

In [79]:
weights_normalized1e7 = weights1e7 / norms
weights_normalized1e8 = weights1e8 / norms
weights_normalized1e4 = weights1e4 / norms
print weights_normalized1e7[3]

161.317456248


To check your results, if you call `normalized_weights1e7` the normalized version of `weights1e7`, then:
```
print normalized_weights1e7[3]
```
should return 161.31745624837794.

## Evaluating each of the learned models on the test data

Let us now evaluate the three models on the ** test ** data:

In [80]:
(test_feature_matrix, test_output) = get_numpy_data(test_data, all_features, 'price')

Compute the RSS of each of the three normalized weights on the (unnormalized) `test_feature_matrix`:

In [84]:
print compute_rss(test_output, test_feature_matrix, weights_normalized1e7)

2.75962079909e+14


In [85]:
print compute_rss(test_output, test_feature_matrix, weights_normalized1e8)

5.37166150034e+14


In [86]:
print compute_rss(test_output, test_feature_matrix, weights_normalized1e4)

2.2778100476e+14


**Analysis Question No. 8:** Which model performed best on the ** test ** data?